In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import os
import datetime as dt

In [3]:
def preprocess_data(df):
    # Convert the event_time column to a datetime format
    df['event_time'] = pd.to_datetime(df['event_time'])
    # Extract the date and timeslot information from the event_time column
    df['date'] = df['event_time'].dt.date
    df['hour'] = df['event_time'].dt.hour
    df['timeslot'] = pd.cut(df['hour'], 
                         bins=[1, 9, 17, 21, 24], 
                         labels=[0, 1, 2, 3], 
                         include_lowest=True, 
                         right=False)
    
    # Modify the 'date' column for the rows where 'hour' is 0
    # so that 0 counts as last timeslot for the day before
    mask = (df['hour'] == 0)
    df.loc[mask, 'date'] = df.loc[mask, 'event_time'].dt.date - dt.timedelta(days=1)
    df.loc[mask, 'timeslot'] = 3
    
    # Extract week number and day number from date
    df['week'] = pd.to_datetime(df['date']).dt.week
    df['day'] = pd.to_datetime(df['date']).dt.day
    
    #Extract time as week day timeslot
    #ex. if week = 13, day = 4 timeslot = 1 then time = 130401
    df['time'] = df['week'].astype(str).str.zfill(2) + \
             df['day'].astype(str).str.zfill(2) + \
             df['timeslot'].astype(str)
    
    df.set_index('user_id', inplace=True)
    
    return df[["hour", "week", "day","timeslot", "time"]]

In [22]:
import pandas as pd
import datetime as dt

def preprocess_data2(df):
    # Convert the event_time column to a datetime format
    df['event_time'] = pd.to_datetime(df['event_time'], unit='s')

    # Extract the date and timeslot information from the event_time column
    df['date'] = df['event_time'].dt.date
    df['hour'] = df['event_time'].dt.hour
    df['timeslot'] = pd.cut(df['hour'], 
                         bins=[1, 9, 17, 21, 24], 
                         labels=[0, 1, 2, 3], 
                         include_lowest=True, 
                         right=False)
    
    # Modify the 'date' column for the rows where 'hour' is 0
    # so that 0 counts as the last timeslot for the previous day
    mask = (df['hour'] == 0)
    df.loc[mask, 'date'] = df.loc[mask, 'event_time'].dt.date - dt.timedelta(days=1)
    df.loc[mask, 'timeslot'] = 3
    
    # Extract week number and day number from date
    df['week'] = pd.to_datetime(df['date']).dt.isocalendar().week
    df['day'] = pd.to_datetime(df['date']).dt.day
    
    # Extract time as week day timeslot
    # ex. if week = 13, day = 4, timeslot = 1, then time = 130401
    df['time'] = df['week'].astype(str).str.zfill(2) + \
                 df['day'].astype(str).str.zfill(2) + \
                 df['timeslot'].astype(str)
    
    df = df[["user_id", "hour", "week", "day", "timeslot", "time", "played_duration"]]
    df.set_index('user_id', inplace=True)
    return df


In [5]:
def extract_sessions(frame): 
    df_extracted_sessions = []
    for val in set(frame['time']):
        print ('Extracting frequency counts for time=%s' % val)
        tmp_df = frame['time'].eq(val).groupby(frame.index).sum()
        tmp_df.name = 'time=%s' % val
        df_extracted_sessions.append(tmp_df)
    return pd.concat(df_extracted_sessions, axis=1)

# KKTV 17.11 DATA

In [4]:
import os
import pandas as pd

dir_path = 'C:/Users/Aleni/OneDrive/Skrivbord/KKTV-Game-17.11-master/public' # replace with the path to your data files
num_files = 75  # replace with the total number of data files
num_parts = 7  # split the data into 7 parts

for part in range(1, num_parts+1):
    # determine the range of files to read for this part
    start = (part-1) * (num_files // num_parts) + 1
    end = start + (num_files // num_parts) - 1
    if part == num_parts:
        end = num_files  # include remaining files in last part

    # create an empty list to hold the DataFrames
    df_list = []

    # loop through CSV files in this range and append them to the list
    for i in range(start, end+1):
        filename = 'data-' + str(i).zfill(3) + '.csv'
        filepath = os.path.join(dir_path, filename)
        df = pd.read_csv(filepath)
        df_list.append(df)

    # concatenate all DataFrames in the list into a single DataFrame
    df = pd.concat(df_list, ignore_index=True)

    # preprocess the data and extract sessions
    print('Processing part {} of {}: Preprocessing...'.format(part, num_parts))
    df = preprocess_data(df)
    df = extract_sessions(df)

    # clean up column names and sort index and columns
    df.columns = df.columns.str[5:]
    df = df.rename(columns=int).sort_index(axis=1)
    df.index = df.index.map(int)
    df.sort_index(axis=0, inplace=True)

    # save the result to file
    print('Processing part {} of {}: Saving to file...'.format(part, num_parts))
    filename = 'df_part{}_of_{}.csv'.format(part, num_parts)
    filepath = os.path.join(dir_path, filename)
    df.to_csv(filepath, index=True)
    
print('Done!')

Processing part 1 of 7: Preprocessing...


C:\Users\Aleni\AppData\Local\Temp\ipykernel_20648\1144975981.py:20: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = pd.to_datetime(df['date']).dt.week


Extracting frequency counts for time=32123
Extracting frequency counts for time=29201
Extracting frequency counts for time=03203
Extracting frequency counts for time=16192
Extracting frequency counts for time=28103
Extracting frequency counts for time=31011
Extracting frequency counts for time=31053
Extracting frequency counts for time=30252
Extracting frequency counts for time=27070
Extracting frequency counts for time=12222
Extracting frequency counts for time=33163
Extracting frequency counts for time=33193
Extracting frequency counts for time=31043
Extracting frequency counts for time=07150
Extracting frequency counts for time=03211
Extracting frequency counts for time=24153
Extracting frequency counts for time=22033
Extracting frequency counts for time=29202
Extracting frequency counts for time=23090
Extracting frequency counts for time=17292
Extracting frequency counts for time=05010
Extracting frequency counts for time=12211
Extracting frequency counts for time=14041
Extracting 

Extracting frequency counts for time=03221
Extracting frequency counts for time=20201
Extracting frequency counts for time=31313
Extracting frequency counts for time=27081
Extracting frequency counts for time=10113
Extracting frequency counts for time=25250
Extracting frequency counts for time=16221
Extracting frequency counts for time=08231
Extracting frequency counts for time=22020
Extracting frequency counts for time=23111
Extracting frequency counts for time=14081
Extracting frequency counts for time=31021
Extracting frequency counts for time=32122
Extracting frequency counts for time=31012
Extracting frequency counts for time=19133
Extracting frequency counts for time=07132
Extracting frequency counts for time=30272
Extracting frequency counts for time=18060
Extracting frequency counts for time=12242
Extracting frequency counts for time=02103
Extracting frequency counts for time=12233
Extracting frequency counts for time=02091
Extracting frequency counts for time=12201
Extracting 

Extracting frequency counts for time=16203
Extracting frequency counts for time=05303
Extracting frequency counts for time=29222
Extracting frequency counts for time=12260
Extracting frequency counts for time=28160
Extracting frequency counts for time=13013
Extracting frequency counts for time=23061
Extracting frequency counts for time=33160
Extracting frequency counts for time=23100
Extracting frequency counts for time=14091
Extracting frequency counts for time=24171
Extracting frequency counts for time=01070
Extracting frequency counts for time=27083
Extracting frequency counts for time=04291
Extracting frequency counts for time=09021
Extracting frequency counts for time=14040
Extracting frequency counts for time=01083
Extracting frequency counts for time=03161
Extracting frequency counts for time=26270
Extracting frequency counts for time=33173
Extracting frequency counts for time=24161
Extracting frequency counts for time=14063
Extracting frequency counts for time=08203
Extracting 

...

Extracting frequency counts for time=32123
Extracting frequency counts for time=29201
Extracting frequency counts for time=03203
Extracting frequency counts for time=16192
Extracting frequency counts for time=28103
Extracting frequency counts for time=31011
Extracting frequency counts for time=31053
Extracting frequency counts for time=30252
Extracting frequency counts for time=27070
Extracting frequency counts for time=12222
Extracting frequency counts for time=31043
Extracting frequency counts for time=07150
Extracting frequency counts for time=03211
Extracting frequency counts for time=24153
Extracting frequency counts for time=22033
Extracting frequency counts for time=29202
Extracting frequency counts for time=23090
Extracting frequency counts for time=05010
Extracting frequency counts for time=17292
Extracting frequency counts for time=12211
Extracting frequency counts for time=14041
Extracting frequency counts for time=14043
Extracting frequency counts for time=05313
Extracting 

Extracting frequency counts for time=08231
Extracting frequency counts for time=22020
Extracting frequency counts for time=23111
Extracting frequency counts for time=14081
Extracting frequency counts for time=31021
Extracting frequency counts for time=31012
Extracting frequency counts for time=32122
Extracting frequency counts for time=19133
Extracting frequency counts for time=12242
Extracting frequency counts for time=30272
Extracting frequency counts for time=18060
Extracting frequency counts for time=07132
Extracting frequency counts for time=02103
Extracting frequency counts for time=12233
Extracting frequency counts for time=02091
Extracting frequency counts for time=12201
Extracting frequency counts for time=13273
Extracting frequency counts for time=23112
Extracting frequency counts for time=18013
Extracting frequency counts for time=14031
Extracting frequency counts for time=21242
Extracting frequency counts for time=13301
Extracting frequency counts for time=07161
Extracting 

Extracting frequency counts for time=14040
Extracting frequency counts for time=01083
Extracting frequency counts for time=03161
Extracting frequency counts for time=26270
Extracting frequency counts for time=24161
Extracting frequency counts for time=14063
Extracting frequency counts for time=08203
Extracting frequency counts for time=25221
Extracting frequency counts for time=32081
Extracting frequency counts for time=25231
Extracting frequency counts for time=19113
Extracting frequency counts for time=24152
Extracting frequency counts for time=12261
Extracting frequency counts for time=22032
Extracting frequency counts for time=27080
Extracting frequency counts for time=16213
Extracting frequency counts for time=31062
Extracting frequency counts for time=14080
Extracting frequency counts for time=16182
Extracting frequency counts for time=32101
Extracting frequency counts for time=26282
Extracting frequency counts for time=23060
Extracting frequency counts for time=16193
Extracting 

Extracting frequency counts for time=27053
Extracting frequency counts for time=28111
Extracting frequency counts for time=31013
Extracting frequency counts for time=10071
Extracting frequency counts for time=25252
Extracting frequency counts for time=05020
Extracting frequency counts for time=27071
Extracting frequency counts for time=28110
Extracting frequency counts for time=28120
Extracting frequency counts for time=26283
Extracting frequency counts for time=10070
Extracting frequency counts for time=31060
Extracting frequency counts for time=03191
Extracting frequency counts for time=07162
Extracting frequency counts for time=24143
Extracting frequency counts for time=07131
Extracting frequency counts for time=10090
Extracting frequency counts for time=30273
Extracting frequency counts for time=11152
Extracting frequency counts for time=10092
Extracting frequency counts for time=11172
Extracting frequency counts for time=05033
Extracting frequency counts for time=29192
Extracting 

Extracting frequency counts for time=08233
Extracting frequency counts for time=07140
Extracting frequency counts for time=18021
Extracting frequency counts for time=32130
Extracting frequency counts for time=28102
Extracting frequency counts for time=07141
Extracting frequency counts for time=08251
Extracting frequency counts for time=15120
Extracting frequency counts for time=15123
Extracting frequency counts for time=16233
Extracting frequency counts for time=16191
Extracting frequency counts for time=32131
Extracting frequency counts for time=07182
Extracting frequency counts for time=08222
Extracting frequency counts for time=21241
Extracting frequency counts for time=18061
Extracting frequency counts for time=02130
Extracting frequency counts for time=05013
Extracting frequency counts for time=13021
Extracting frequency counts for time=16210
Extracting frequency counts for time=11160
Extracting frequency counts for time=27032
Extracting frequency counts for time=04241
Extracting 

# test_source_events

In [12]:
df = pd.read_csv("test_source_events.csv")
df

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,title_in_simulcast,internet_connection_type
0,30460,139780,4203642,134,1.644848e+09,919,0,0,1,1
1,30460,139780,4203766,134,1.644854e+09,1,0,0,1,1
2,30460,139780,4206734,331,1.645010e+09,22,0,0,1,0
3,30460,139780,4217713,331,1.645705e+09,1220,1,0,1,0
4,30460,139780,4193024,398,1.646645e+09,9,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2382746,38075,174948,5252063,625,1.662919e+09,4886,1,1,0,2
2382747,38075,174948,5252063,625,1.662919e+09,5,0,1,0,2
2382748,38075,174948,5252063,625,1.662919e+09,1,0,1,0,2
2382749,38075,174948,5252064,625,1.663256e+09,4823,2,1,0,2


In [13]:
df = preprocess_data2(df)
df

C:\Users\Aleni\AppData\Local\Temp\ipykernel_29656\2956455478.py:21: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = pd.to_datetime(df['date']).dt.week


,hour,week,day,timeslot,time
user_id,,,,,
30460,14,7,14,1,07141
30460,15,7,14,1,07141
30460,11,7,16,1,07161
30460,12,8,24,1,08241
30460,9,10,7,1,10071
...,...,...,...,...,...
38075,17,36,11,2,36112
38075,17,36,11,2,36112
38075,17,36,11,2,36112


In [14]:
df = extract_sessions(df)

# clean up column names and sort index and columns
df.columns = df.columns.str[5:]
df = df.rename(columns=int).sort_index(axis=1)
df.index = df.index.map(int)
df.sort_index(axis=0, inplace=True)

# save the result to file
df.to_csv("Procecssed_test_source_events", index=True)

Extracting frequency counts for time=29201
Extracting frequency counts for time=10103
Extracting frequency counts for time=13313
Extracting frequency counts for time=12271
Extracting frequency counts for time=18061
Extracting frequency counts for time=19153
Extracting frequency counts for time=19151
Extracting frequency counts for time=37143
Extracting frequency counts for time=16211
Extracting frequency counts for time=33180
Extracting frequency counts for time=05051
Extracting frequency counts for time=07150
Extracting frequency counts for time=07151
Extracting frequency counts for time=08211
Extracting frequency counts for time=13302
Extracting frequency counts for time=15162
Extracting frequency counts for time=18020
Extracting frequency counts for time=20192
Extracting frequency counts for time=21281
Extracting frequency counts for time=03173
Extracting frequency counts for time=25203
Extracting frequency counts for time=19100
Extracting frequency counts for time=08210
Extracting 

Extracting frequency counts for time=16202
Extracting frequency counts for time=16221
Extracting frequency counts for time=10112
Extracting frequency counts for time=11191
Extracting frequency counts for time=30261
Extracting frequency counts for time=01063
Extracting frequency counts for time=14071
Extracting frequency counts for time=02110
Extracting frequency counts for time=21251
Extracting frequency counts for time=27100
Extracting frequency counts for time=05020
Extracting frequency counts for time=11182
Extracting frequency counts for time=31063
Extracting frequency counts for time=25221
Extracting frequency counts for time=24191
Extracting frequency counts for time=37121
Extracting frequency counts for time=33171
Extracting frequency counts for time=06120
Extracting frequency counts for time=07142
Extracting frequency counts for time=34231
Extracting frequency counts for time=10121
Extracting frequency counts for time=19102
Extracting frequency counts for time=08222
Extracting 

Extracting frequency counts for time=24181
Extracting frequency counts for time=19091
Extracting frequency counts for time=14043
Extracting frequency counts for time=34270
Extracting frequency counts for time=07192
Extracting frequency counts for time=10111
Extracting frequency counts for time=15122
Extracting frequency counts for time=13011
Extracting frequency counts for time=36083
Extracting frequency counts for time=04303
Extracting frequency counts for time=10131
Extracting frequency counts for time=08273
Extracting frequency counts for time=12252
Extracting frequency counts for time=01090
Extracting frequency counts for time=21270
Extracting frequency counts for time=30282
Extracting frequency counts for time=20191
Extracting frequency counts for time=36111
Extracting frequency counts for time=16182
Extracting frequency counts for time=25212
Extracting frequency counts for time=34232
Extracting frequency counts for time=16232
Extracting frequency counts for time=33170
Extracting 

Extracting frequency counts for time=02150
Extracting frequency counts for time=35311
Extracting frequency counts for time=36053
Extracting frequency counts for time=35040
Extracting frequency counts for time=14060
Extracting frequency counts for time=17282
Extracting frequency counts for time=03172
Extracting frequency counts for time=22031
Extracting frequency counts for time=32091
Extracting frequency counts for time=18032
Extracting frequency counts for time=28120
Extracting frequency counts for time=30263
Extracting frequency counts for time=01083
Extracting frequency counts for time=24183
Extracting frequency counts for time=23100
Extracting frequency counts for time=34242
Extracting frequency counts for time=34243
Extracting frequency counts for time=19130
Extracting frequency counts for time=12261
Extracting frequency counts for time=28112
Extracting frequency counts for time=02113
Extracting frequency counts for time=23101
Extracting frequency counts for time=16192
Extracting 

Extracting frequency counts for time=27080
Extracting frequency counts for time=29180
Extracting frequency counts for time=27083
Extracting frequency counts for time=20222
Extracting frequency counts for time=08223
Extracting frequency counts for time=19140
Extracting frequency counts for time=30260
Extracting frequency counts for time=23071
Extracting frequency counts for time=26292
Extracting frequency counts for time=06091
Extracting frequency counts for time=33213
Extracting frequency counts for time=03183
Extracting frequency counts for time=27043
Extracting frequency counts for time=17263
Extracting frequency counts for time=13023
Extracting frequency counts for time=32122
Extracting frequency counts for time=23081
Extracting frequency counts for time=27071
Extracting frequency counts for time=03203
Extracting frequency counts for time=35023
Extracting frequency counts for time=14072
Extracting frequency counts for time=30312
Extracting frequency counts for time=32081
Extracting 

Extracting frequency counts for time=30302
Extracting frequency counts for time=07202
Extracting frequency counts for time=25223
Extracting frequency counts for time=33162
Extracting frequency counts for time=12272
Extracting frequency counts for time=23073
Extracting frequency counts for time=13293
Extracting frequency counts for time=02112
Extracting frequency counts for time=15141
Extracting frequency counts for time=16242
Extracting frequency counts for time=19143
Extracting frequency counts for time=21283
Extracting frequency counts for time=29221
Extracting frequency counts for time=37131
Extracting frequency counts for time=02141
Extracting frequency counts for time=25242
Extracting frequency counts for time=11142
Extracting frequency counts for time=18021
Extracting frequency counts for time=01072
Extracting frequency counts for time=03221
Extracting frequency counts for time=19141
Extracting frequency counts for time=16181
Extracting frequency counts for time=20162
Extracting 

## New Idea - instead of nr of sessions -> active time

In [23]:
test_df = pd.read_csv("test_source_events.csv")
train_df = pd.read_csv("train_source_events.csv")

In [24]:
test_df = preprocess_data2(test_df)
train_df = preprocess_data2(train_df)

In [35]:
test_df

,hour,week,day,timeslot,time,played_duration
user_id,,,,,,
30460,14,7,14,1,07141,919
30460,15,7,14,1,07141,1
30460,11,7,16,1,07161,22
30460,12,8,24,1,08241,1220
30460,9,10,7,1,10071,9
...,...,...,...,...,...,...
38075,17,36,11,2,36112,4886
38075,17,36,11,2,36112,5
38075,17,36,11,2,36112,1


In [36]:
def extract_sessions(frame):
    df_extracted_sessions = []
    for val in set(frame['time']):
        print('Extracting duration sums for time=%s' % val)
        tmp_df = frame[frame['time'] == val].reset_index().groupby('user_id')['played_duration'].sum()
        tmp_df.name = 'duration_sum_time=%s' % val
        df_extracted_sessions.append(tmp_df)
    return pd.concat(df_extracted_sessions, axis=1)


In [50]:
data_test_df = extract_sessions(test_df)

Extracting duration sums for time=28170
Extracting duration sums for time=35033
Extracting duration sums for time=16213
Extracting duration sums for time=29221
Extracting duration sums for time=14083
Extracting duration sums for time=30280
Extracting duration sums for time=22041
Extracting duration sums for time=26291
Extracting duration sums for time=06101
Extracting duration sums for time=12231
Extracting duration sums for time=23060
Extracting duration sums for time=34272
Extracting duration sums for time=03201
Extracting duration sums for time=33163
Extracting duration sums for time=12272
Extracting duration sums for time=13300
Extracting duration sums for time=27052
Extracting duration sums for time=12212
Extracting duration sums for time=37141
Extracting duration sums for time=30311
Extracting duration sums for time=24141
Extracting duration sums for time=01072
Extracting duration sums for time=12250
Extracting duration sums for time=31052
Extracting duration sums for time=13022


Extracting duration sums for time=18072
Extracting duration sums for time=06092
Extracting duration sums for time=18082
Extracting duration sums for time=18032
Extracting duration sums for time=27040
Extracting duration sums for time=27061
Extracting duration sums for time=32102
Extracting duration sums for time=22040
Extracting duration sums for time=22303
Extracting duration sums for time=21231
Extracting duration sums for time=04242
Extracting duration sums for time=06091
Extracting duration sums for time=01050
Extracting duration sums for time=03191
Extracting duration sums for time=21253
Extracting duration sums for time=31010
Extracting duration sums for time=13280
Extracting duration sums for time=13281
Extracting duration sums for time=10122
Extracting duration sums for time=15131
Extracting duration sums for time=01052
Extracting duration sums for time=17261
Extracting duration sums for time=26273
Extracting duration sums for time=17251
Extracting duration sums for time=01083


Extracting duration sums for time=17011
Extracting duration sums for time=28130
Extracting duration sums for time=06130
Extracting duration sums for time=03172
Extracting duration sums for time=23122
Extracting duration sums for time=32120
Extracting duration sums for time=25210
Extracting duration sums for time=33182
Extracting duration sums for time=29240
Extracting duration sums for time=20180
Extracting duration sums for time=07153
Extracting duration sums for time=32113
Extracting duration sums for time=07191
Extracting duration sums for time=06103
Extracting duration sums for time=29182
Extracting duration sums for time=23061
Extracting duration sums for time=12241
Extracting duration sums for time=35043
Extracting duration sums for time=29223
Extracting duration sums for time=17253
Extracting duration sums for time=28113
Extracting duration sums for time=08272
Extracting duration sums for time=33203
Extracting duration sums for time=03230
Extracting duration sums for time=30313


Extracting duration sums for time=35010
Extracting duration sums for time=16192
Extracting duration sums for time=10113
Extracting duration sums for time=06090
Extracting duration sums for time=15162
Extracting duration sums for time=13290
Extracting duration sums for time=34273
Extracting duration sums for time=36080
Extracting duration sums for time=08230
Extracting duration sums for time=37122
Extracting duration sums for time=10073
Extracting duration sums for time=16203
Extracting duration sums for time=08243
Extracting duration sums for time=30261
Extracting duration sums for time=26271
Extracting duration sums for time=11180
Extracting duration sums for time=10093
Extracting duration sums for time=26302
Extracting duration sums for time=15152
Extracting duration sums for time=34222
Extracting duration sums for time=20183
Extracting duration sums for time=31032
Extracting duration sums for time=11192
Extracting duration sums for time=08273
Extracting duration sums for time=32081


Extracting duration sums for time=09043
Extracting duration sums for time=23110
Extracting duration sums for time=01081
Extracting duration sums for time=05043
Extracting duration sums for time=30312
Extracting duration sums for time=22053
Extracting duration sums for time=33151
Extracting duration sums for time=25243
Extracting duration sums for time=18042
Extracting duration sums for time=33161
Extracting duration sums for time=05061
Extracting duration sums for time=06122
Extracting duration sums for time=07190
Extracting duration sums for time=07150
Extracting duration sums for time=07203
Extracting duration sums for time=32091
Extracting duration sums for time=19123
Extracting duration sums for time=18023
Extracting duration sums for time=14052
Extracting duration sums for time=22022
Extracting duration sums for time=35042
Extracting duration sums for time=27101
Extracting duration sums for time=32110
Extracting duration sums for time=22010
Extracting duration sums for time=06133


Extracting duration sums for time=33192
Extracting duration sums for time=26292
Extracting duration sums for time=31062
Extracting duration sums for time=31053
Extracting duration sums for time=13312
Extracting duration sums for time=27042
Extracting duration sums for time=01071


In [53]:
data_test_df

,duration_sum_time=28170,duration_sum_time=35033,duration_sum_time=16213,duration_sum_time=29221,duration_sum_time=14083,duration_sum_time=30280,duration_sum_time=22041,duration_sum_time=26291,duration_sum_time=06101,duration_sum_time=12231,...,duration_sum_time=07202,duration_sum_time=04261,duration_sum_time=30300,duration_sum_time=33192,duration_sum_time=26292,duration_sum_time=31062,duration_sum_time=31053,duration_sum_time=13312,duration_sum_time=27042,duration_sum_time=01071
user_id,,,,,,,,,,,,,,,,,,,,,
30469,2833.0,NaN,NaN,3690.0,NaN,1718.0,NaN,NaN,3188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,415.0
30473,2733.0,NaN,NaN,162.0,NaN,NaN,4354.0,4437.0,238.0,NaN,...,NaN,NaN,1364.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30481,12.0,NaN,NaN,NaN,NaN,NaN,NaN,1420.0,NaN,1766.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30502,3955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30504,5349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# clean up column names and sort index and columns
df = data_test_df

# Extract integer values from column names
column_names = df.columns
integer_values = [int(name.split('=')[1]) for name in column_names]

# Rename the columns with the extracted integer values
df.columns = integer_values

# Sort the columns based on their integer values
df = df.reindex(sorted(df.columns), axis=1)

# Display the updated DataFrame
print(df)

         1030   1031   1032   1033   1040    1041   1042   1043    1050    
user_id                                                                    
30469      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN  \
30473      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
30481      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
30502      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
30504      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN  1253.0   
...        ...    ...    ...    ...    ...     ...    ...    ...     ...   
34161      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
31225      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
33015      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
36582      NaN    NaN    NaN    NaN    NaN     NaN    NaN    NaN     NaN   
36856      NaN    NaN    NaN    NaN    NaN  1775.0    NaN    NaN     NaN   

         10

In [55]:
# Replace NaN values with zeros
df.fillna(0, inplace=True)

In [56]:
df

,1030,1031,1032,1033,1040,1041,1042,1043,1050,1051,...,37162,37163,52010,52011,52012,52013,52020,52021,52022,52023
user_id,,,,,,,,,,,,,,,,,,,,,
30469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1889.0,0.0,0.0,0.0
30473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3727.0,0.0,0.0,0.0,0.0,0.0,0.0
30504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1253.0,0.0,...,1738.0,0.0,6381.0,2730.0,0.0,3276.0,3766.0,3399.0,0.0,1664.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
# save the result to file
df.to_csv("Time_instead_of_events_test_df.csv", index=True)

In [60]:
#train df
data_train_df = extract_sessions(train_df)

Extracting duration sums for time=28170
Extracting duration sums for time=35033
Extracting duration sums for time=16213
Extracting duration sums for time=29221
Extracting duration sums for time=14083
Extracting duration sums for time=30280
Extracting duration sums for time=22041
Extracting duration sums for time=26291
Extracting duration sums for time=06101
Extracting duration sums for time=12231
Extracting duration sums for time=23060
Extracting duration sums for time=34272
Extracting duration sums for time=03201
Extracting duration sums for time=33163
Extracting duration sums for time=12272
Extracting duration sums for time=13300
Extracting duration sums for time=12212
Extracting duration sums for time=27052
Extracting duration sums for time=37141
Extracting duration sums for time=30311
Extracting duration sums for time=24141
Extracting duration sums for time=01072
Extracting duration sums for time=12250
Extracting duration sums for time=31052
Extracting duration sums for time=13022


Extracting duration sums for time=18072
Extracting duration sums for time=06092
Extracting duration sums for time=18082
Extracting duration sums for time=18032
Extracting duration sums for time=27040
Extracting duration sums for time=27061
Extracting duration sums for time=32102
Extracting duration sums for time=22040
Extracting duration sums for time=22303
Extracting duration sums for time=21231
Extracting duration sums for time=04242
Extracting duration sums for time=01050
Extracting duration sums for time=06091
Extracting duration sums for time=03191
Extracting duration sums for time=21253
Extracting duration sums for time=31010
Extracting duration sums for time=13280
Extracting duration sums for time=13281
Extracting duration sums for time=10122
Extracting duration sums for time=15131
Extracting duration sums for time=01052
Extracting duration sums for time=26273
Extracting duration sums for time=17261
Extracting duration sums for time=17251
Extracting duration sums for time=01083


Extracting duration sums for time=04243
Extracting duration sums for time=09041
Extracting duration sums for time=17011
Extracting duration sums for time=06130
Extracting duration sums for time=28130
Extracting duration sums for time=03172
Extracting duration sums for time=23122
Extracting duration sums for time=32120
Extracting duration sums for time=25210
Extracting duration sums for time=33182
Extracting duration sums for time=29240
Extracting duration sums for time=20180
Extracting duration sums for time=07153
Extracting duration sums for time=32113
Extracting duration sums for time=07191
Extracting duration sums for time=06103
Extracting duration sums for time=29182
Extracting duration sums for time=23061
Extracting duration sums for time=12241
Extracting duration sums for time=35043
Extracting duration sums for time=29223
Extracting duration sums for time=17253
Extracting duration sums for time=03230
Extracting duration sums for time=28113
Extracting duration sums for time=08272


Extracting duration sums for time=35030
Extracting duration sums for time=25242
Extracting duration sums for time=35010
Extracting duration sums for time=16192
Extracting duration sums for time=10113
Extracting duration sums for time=06090
Extracting duration sums for time=15162
Extracting duration sums for time=13290
Extracting duration sums for time=34273
Extracting duration sums for time=36080
Extracting duration sums for time=08230
Extracting duration sums for time=37122
Extracting duration sums for time=10073
Extracting duration sums for time=16203
Extracting duration sums for time=08243
Extracting duration sums for time=26271
Extracting duration sums for time=30261
Extracting duration sums for time=11180
Extracting duration sums for time=10093
Extracting duration sums for time=26302
Extracting duration sums for time=15152
Extracting duration sums for time=34222
Extracting duration sums for time=20183
Extracting duration sums for time=31032
Extracting duration sums for time=11192


Extracting duration sums for time=35020
Extracting duration sums for time=19140
Extracting duration sums for time=25252
Extracting duration sums for time=09043
Extracting duration sums for time=23110
Extracting duration sums for time=01081
Extracting duration sums for time=05043
Extracting duration sums for time=30312
Extracting duration sums for time=22053
Extracting duration sums for time=25243
Extracting duration sums for time=33151
Extracting duration sums for time=18042
Extracting duration sums for time=33161
Extracting duration sums for time=05061
Extracting duration sums for time=07203
Extracting duration sums for time=07190
Extracting duration sums for time=07150
Extracting duration sums for time=06122
Extracting duration sums for time=32091
Extracting duration sums for time=18023
Extracting duration sums for time=19123
Extracting duration sums for time=14052
Extracting duration sums for time=22022
Extracting duration sums for time=35042
Extracting duration sums for time=22010


Extracting duration sums for time=30300
Extracting duration sums for time=33192
Extracting duration sums for time=31062
Extracting duration sums for time=31053
Extracting duration sums for time=13312
Extracting duration sums for time=11140
Extracting duration sums for time=01071


In [61]:
# clean up column names and sort index and columns
df = data_train_df

# Extract integer values from column names
column_names = df.columns
integer_values = [int(name.split('=')[1]) for name in column_names]

# Rename the columns with the extracted integer values
df.columns = integer_values

# Sort the columns based on their integer values
df = df.reindex(sorted(df.columns), axis=1)

# Replace NaN values with zeros
df.fillna(0, inplace=True)

print(df)

         1030    1031   1032    1033   1040    1041   1042   1043    1050    
user_id                                                                      
0          0.0     0.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0  \
9          0.0     0.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0   
16         0.0  4404.0    0.0  2787.0    0.0  1278.0    0.0    0.0  3602.0   
17         0.0  4844.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0   
20         0.0     0.0    0.0     0.0    0.0    18.0    0.0    0.0     0.0   
...        ...     ...    ...     ...    ...     ...    ...    ...     ...   
20790      0.0     0.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0   
9440       0.0     0.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0   
17197      0.0     0.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0   
10676      0.0     0.0    0.0     0.0    0.0     0.0    0.0    0.0     0.0   
11832      0.0     0.0    0.0     0.0    0.0     0.0    0.0    0

In [62]:
# save the result to file
df.to_csv("Time_instead_of_events_train_df.csv", index=True)